In [1]:
import pandas as pd
import os
root_dir = os.getcwd()
from data_loading import load_ftn, load_stadium_data, load_weather_data
from data_cleaning import feature_engineering


In [2]:
df_games=pd.read_csv('data/games.csv')
    
df_games=pd.merge(df_games, load_stadium_data(),left_on='gameId', right_on='old_game_id', how='left')
df_games=pd.merge(df_games, load_weather_data(),on='gameId', how='left')

2022 done.
Downcasting floats.


In [3]:
df_plays = feature_engineering(pd.read_csv('data/plays.csv'))

In [4]:
%%capture
ftn_merged = load_ftn()

In [5]:
merged_id_df = pd.read_csv('data/merged_ids.csv')

In [6]:
xp_fname = os.path.join(root_dir, "data/team_xpass_ratios.csv")
pr_fname = os.path.join(root_dir, "data/team_pass_rates.csv")
cf_fname = os.path.join(root_dir, "data/coverages_faced.csv")
cu_fname = os.path.join(root_dir, 'data/coverages_used.csv')

In [7]:
xp_df = pd.read_csv(xp_fname).drop(columns='Unnamed: 0')
pr_df = pd.read_csv(pr_fname).drop(columns='Unnamed: 0')

In [8]:
xp_df.head(3)

,gameId,playId,off_xpass,def_xpass
0,2022091110,347,0.58,0.58
1,2022091110,400,0.58,0.58
2,2022091110,729,0.58,0.58


In [9]:
xp_df.isna().sum()

gameId       0
playId       0
off_xpass    0
def_xpass    0
dtype: int64

In [11]:
merged_base = merged_id_df.merge(df_games[['gameId','week']].drop_duplicates(),how='left',on=['gameId'])

In [12]:
merged_base = merged_base.merge(df_plays[['gameId','playId',
                                          'possessionTeam','defensiveTeam']].drop_duplicates(),
                                how='left',on=['gameId','playId'])

In [13]:
merged_base.head(3)

,gameId,playId,week,possessionTeam,defensiveTeam
0,2022090800,56,1,BUF,LA
1,2022090800,80,1,BUF,LA
2,2022090800,101,1,BUF,LA


In [14]:
mb_w1 = merged_base[merged_base['week'] == 1]

In [15]:
mb_w2_on = merged_base[merged_base['week'] > 1]

In [16]:
w1_ids = mb_w1[['gameId','playId','week','possessionTeam','defensiveTeam']]

In [17]:
w2_on_ids = mb_w2_on[['gameId','playId','week','possessionTeam','defensiveTeam']]

# Work on data_21

Note to do ftn data later maybe, as to not mess up id's (probably has no effect on id's )

In [19]:
cov_21 = pd.read_csv('data_21/cov_21.csv')
cov_21.head(2)

,possessionTeam,Man_def,Zone_def,cover_0_def,cover_1_def,cover_2_def,cover_3_def,Quarters_def,cover_6_def,Other_def,Man_off,Zone_off,cover_0_off,cover_1_off,cover_2_off,cover_3_off,Quarters_off,cover_6_off,Other_off
0,ARI,9.529412,23.647059,3.117647,5.352941,4.000000,12.588235,4.117647,2.882353,1.117647,7.411765,28.352941,1.823529,5.588235,4.588235,14.000000,5.882353,3.705882,0.176471
1,ATL,11.117647,22.000000,1.647059,9.000000,12.294118,8.117647,0.705882,0.647059,0.705882,8.058824,24.705882,1.235294,6.294118,5.588235,11.294118,5.882353,1.823529,0.647059


In [20]:
team_pr_21 = pd.read_csv('data_21/team_pr_21.csv')
team_pr_21.head(2)

,possessionTeam,pass_rate_def,pass_rate_off
0,ARI,0.573372,0.573464
1,ATL,0.548059,0.603575


# Calculate team pass rates

Note: use team pass rate from last year as xpass for week 1 (due to 2021 data incompleteness)

In [21]:
pr_df.head(1)

,gameId,playId,pass_rate_off,pass_rate_def
0,2022102302,2655,0.654054,0.648794


In [22]:
w1_ids.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam
0,2022090800,56,1,BUF,LA


#### Merge in '21 pass ratios to '22 week one df:

In [23]:
w1_pr = w1_ids.merge(team_pr_21.drop(columns=['pass_rate_off']).rename(columns={'possessionTeam':'defensiveTeam'}),on='defensiveTeam')
w1_pr = w1_pr.merge(team_pr_21.drop(columns=['pass_rate_def']),on='possessionTeam')

In [27]:
w1_pr.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def,pass_rate_off
0,2022090800,56,1,BUF,LA,0.615627,0.619126


We bake in xpass here because it's just going to replicate pass_rate_off/def for week 1

In [28]:
w1_pr['off_xpass'] = w1_pr['pass_rate_off'].copy()
w1_pr['def_xpass'] = w1_pr['pass_rate_def'].copy()

In [29]:
w1_pr.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def,pass_rate_off,off_xpass,def_xpass
0,2022090800,56,1,BUF,LA,0.615627,0.619126,0.619126,0.615627


# Calculate coverage data

Here we just use defensive rates for simplicity, can revisit later

In [32]:
cov_def = cov_21[[x for x in cov_21.columns if '_off' not in x]].rename(columns={'possessionTeam':'defensiveTeam'})
cov_def = cov_def.rename(columns={'cover_2_def':'Cover-2_def','cover_0_def':'Cover-0_def'})

In [33]:
cov_def.head(1)

,defensiveTeam,Man_def,Zone_def,Cover-0_def,cover_1_def,Cover-2_def,cover_3_def,Quarters_def,cover_6_def,Other_def
0,ARI,9.529412,23.647059,3.117647,5.352941,4.0,12.588235,4.117647,2.882353,1.117647


In [34]:
w1_merged = w1_pr.merge(cov_def,how='left',on='defensiveTeam')

In [35]:
w1_merged.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def,pass_rate_off,off_xpass,def_xpass,Man_def,Zone_def,Cover-0_def,cover_1_def,Cover-2_def,cover_3_def,Quarters_def,cover_6_def,Other_def
0,2022090800,56,1,BUF,LA,0.615627,0.619126,0.619126,0.615627,7.117647,37.117647,0.705882,6.352941,0.647059,20.823529,7.647059,7.705882,0.352941


# Integrate '22 coverage data (i.e., for week 2 on)

In [38]:
w2_on_ids.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam
1950,2022091500,55,2,KC,LAC


In [40]:
cu_df = pd.read_csv('data/coverages_used.csv')
cu_df['week'] = cu_df['week'].astype(int)
cu_df.head(1)

,week,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,defensiveTeam
0,2,21.0,2.0,21.0,7.0,4.0,3.0,26.0,4.0,34.0,ARI


In [41]:
cu_w2_on = w2_on_ids.merge(cu_df,how='left',left_on=['defensiveTeam','week'],right_on=['defensiveTeam','week'])

In [42]:
cu_w2_on.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def
0,2022091500,55,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0


# Get data for week 2 onward

In [48]:
df_w2_on = cu_w2_on.merge(pr_df,how='left',on=['gameId','playId'])
df_w2_on = df_w2_on.merge(xp_df,how='left',on=['gameId','playId'])

df_w2_on.head(2)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,pass_rate_off,pass_rate_def,off_xpass,def_xpass
0,2022091500,55,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0,0.625,0.763636,0.686433,0.738111
1,2022091500,76,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0,0.625,0.763636,0.658727,0.738111


In [53]:
w1_merged = w1_merged[df_w2_on.columns]

In [55]:
df_w2_on.week.min()

2

# TODO: find way to make ordering match test ig?

2

# Integrate week 1-imputed, week 2 onward data

where is 'week' lost?

In [58]:
merged_base = pd.concat([w1_merged,df_w2_on],axis=0)

In [68]:
merged_base.sample(3)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,pass_rate_off,pass_rate_def,off_xpass,def_xpass
798,2022091105,2498,1,IND,HOU,8.235294,2.176471,8.058824,2.764706,9.294118,0.588235,8.647059,0.000000,22.470588,0.532609,0.520913,0.532609,0.520913
144,2022091100,1651,1,ATL,NO,9.235294,0.764706,9.941176,6.117647,5.117647,1.294118,12.470588,1.294118,21.294118,0.603575,0.607000,0.603575,0.607000
11504,2022103004,1976,8,TEN,HOU,121.000000,11.000000,68.000000,44.000000,99.000000,1.000000,69.000000,21.000000,275.000000,0.493671,0.557951,0.437450,0.557000


Get cols final for everything to line up later (sans off. coverage feats)

In [69]:
cols_final = ['gameId', 'playId', 'n_offense_backfield', 'n_defense_box',
       'is_no_huddle', 'is_motion', 'pass_rate_off', 'pass_rate_def',
       'off_xpass', 'def_xpass', 'week', 'possessionTeam', 'defensiveTeam',
       'cover_3_def', 'cover_6_def', 'cover_1_def', 'Quarters_def',
       'Cover-2_def', 'Cover-0_def', 'Man_def', 'Other_def', 'Zone_def',
       'off_snaps_lost', 'def_snaps_lost']

In [70]:
inj_fname = os.path.join(root_dir, 'data/snaps_lost_injury.csv')

In [71]:
inj_df = pd.read_csv(inj_fname)
merged_base = merged_base.merge(inj_df.drop(columns=['def_snaps_lost']),how='left',
                   left_on=['possessionTeam','week'], right_on=['club_code','week']).drop(columns=['club_code'])
merged_base = merged_base.merge(inj_df.drop(columns=['off_snaps_lost']),how='left',
                   left_on=['defensiveTeam','week'], right_on=['club_code','week']).drop(columns=['club_code'])

In [72]:
merged_base.isna().sum()

gameId               0
playId               0
week                 0
possessionTeam       0
defensiveTeam        0
cover_3_def          0
cover_6_def          0
cover_1_def          0
Quarters_def         0
Cover-2_def          0
Cover-0_def          0
Man_def              0
Other_def            0
Zone_def             0
pass_rate_off        0
pass_rate_def        0
off_xpass            0
def_xpass            0
off_snaps_lost    5436
def_snaps_lost    5670
dtype: int64

# TODO: sanity check injury snaps lost process, if sound just fillna w/0

In [77]:
merged_base[merged_base.off_snaps_lost.isna()].sample(5)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,pass_rate_off,pass_rate_def,off_xpass,def_xpass,off_snaps_lost,def_snaps_lost
3171,2022091809,3451,2,CIN,DAL,13.000000,4.000000,23.000000,7.000000,5.000000,1.000000,24.000000,3.000000,29.000000,0.688889,0.448276,0.646474,0.450764,NaN,18.0
1742,2022091113,1587,1,DAL,TB,15.117647,2.000000,6.705882,6.823529,8.705882,2.117647,10.823529,2.117647,32.764706,0.597780,0.675629,0.597780,0.675629,NaN,NaN
307,2022091101,1468,1,CLE,CAR,9.882353,1.411765,8.411765,4.235294,3.588235,1.823529,10.529412,0.352941,19.176471,0.538928,0.527068,0.538928,0.527068,NaN,NaN
5281,2022092511,1360,3,ATL,SEA,50.000000,33.000000,8.000000,15.000000,2.000000,3.000000,12.000000,19.000000,100.000000,0.549180,0.703125,0.502538,0.478879,NaN,NaN
9800,2022101602,475,6,NYJ,GB,116.000000,21.000000,41.000000,50.000000,11.000000,4.000000,48.000000,12.000000,198.000000,0.670732,0.540230,0.635240,0.551144,NaN,0.0


# Integrate FTN, injury data